In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils as np_utils
from tensorflow.keras.layers import Input, InputLayer, Conv2D, MaxPool2D, GlobalAveragePooling2D, BatchNormalization, Activation, ReLU, Flatten, Dense, Add, Dropout
from tensorflow.keras.optimizers import  Adam, SGD
from tensorflow.keras.activations import swish
from tensorflow.keras.applications.resnet50 import ResNet50


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## train画像を確認する

train画像の一つを表示してみる

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
im = np.array(Image.open('../input/plant-seedlings-classification/train/Maize/006196e1c.png'))

plt.imshow(im)

## 種類ごとのtrain画像枚数を確認する．

In [ ]:
#植物名のlistを作成
species_list = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]

In [ ]:
import glob

file_num = []
#各ディレクトリの画像枚数をカウントしてlistにする
for i in range(12) :
    imfile = glob.glob('../input/plant-seedlings-classification/train/'+species_list[i]+'/*.png')
    file_num += [len(imfile)]

file_num

棒グラフ化

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (8,5))
ax = fig.add_subplot(111)
ax.bar(species_list,file_num)
plt.xticks(rotation = 90)
plt.show()

## kerasのImageDataGeneratorを用いて学習データの前処理を行う

train画像に対して以下の設定でDataAugmentation(データの水増し)を行う．

In [ ]:
train_generator = ImageDataGenerator(
    #ランダムに回転させる範囲
    rotation_range = 80,
    #ランダムにズームする範囲
    zoom_range = 0.2,
    #ランダムに水平・鉛直方向にシフトさせる範囲
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    #反時計周りのシアー強度
    shear_range = 0.2,
    #水平・鉛直方向にランダムに反転させる
    vertical_flip = True,
    horizontal_flip = True,
)
train_generator = train_generator.flow_from_directory(
        directory = '/kaggle/input/plant-seedlings-classification/train',
        target_size = (299, 299),
        batch_size = 32,
        color_mode = 'rgb',
        #One-hotベクトルでラベルを表現する
        class_mode = "categorical",
        subset ='training'
)

In [ ]:
test_generator = ImageDataGenerator()
test_generator = test_generator.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/',
        classes=['test'],
        target_size=(299, 299),
        batch_size=1,
        color_mode='rgb',
        shuffle=False,
        class_mode="categorical"
)

前処理後の学習用画像を確認

In [ ]:
plt.figure(figsize=(32,32))
for i in range(6):
    batches = next(train_generator)
    # 画像として表示するため、3次元データにし、float から uint8(0-255) にキャストする。
    gen_img = batches[0][i].astype(np.uint8)
    #gen_img = batches[0][i]
    #gen_img = array_to_img(gen_img,scale=True)
    plt.subplot(2, 3, i + 1)
    plt.imshow(gen_img)

In [ ]:
img = train_generator[0][0][1].astype(np.uint8)
#img = array_to_img(img,scale=True)
plt.imshow(img)

## NNのモデルを定義

InceptionResNetV2を使用する．

In [ ]:
input_shape = (299,299,3)

In [ ]:
def ResBlock(x, kernel_size, filter_num) :
    shortcut = x
    
    for l in range(2) :
        x = Conv2D(filter_num, kernel_size, padding='same')(x)
        x = BatchNormalization()(x)
        
        if l == 1 :
            # shortcutとConv層通過後でチャネル数が違うことがあるので1x1convでそろえる
            if K.int_shape(x) != K.int_shape(shortcut) :
                shortcut = Conv2D(filter_num, (1,1), padding='same')(shortcut)
            
            x = Add()([x, shortcut])
        x = Activation('swish')(x)
    
    return x

In [ ]:
block = 5
filter = 64
input = Input(shape=input_shape)
x = Conv2D(filter,(3,3), strides=2, padding='same')(input)
x = BatchNormalization()(x)
x = Activation('swish')(x)

for f in range(block) :
        
    for s in range(3) :
        x = ResBlock(x, (3,3), filter)

    filter = filter * 2
    if f < block - 1 :
        #x = MaxPool2D(pool_size=(2, 2))(x)
        x = Conv2D(filter, (1,1), strides = 2)

#x = Dropout(0.2)(x)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
#x = Dense(256)(x)
#x = ReLU()(x)
output = Dense(12, activation='softmax')(x)

model = Model(inputs=input, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9), metrics=['accuracy'])

In [ ]:
model.summary()

## ニューラルネットワークを学習

In [ ]:
epochs = 30
batch_size = 32
history = model.fit(train_generator, batch_size=batch_size, epochs=epochs, verbose=1)

In [ ]:
train_generator.class_indices

## 提出用ファイルを作成

ネットワークの出力は12次元のベクトルである．また，出力層の活性化関数にsoftmax関数が用いられているため，出力値の範囲は0.0-1.0であり12個の出力を足すと1になるように変換されている．
つまり，ネットワークの出力は12個の植物のどれに分類されるかの確率分布となっている．


In [ ]:
#ネットワークの出力を確認
predict = model_conv.predict(test_generator, steps=test_generator.samples)
predict[0]

In [ ]:
predict = model_conv.predict(test_generator, steps=test_generator.samples)

class_list = []

for i in range(0, predict.shape[0]):
  #最も確率が高いと予測された植物をclass_listに追加する
  y_class = predict[i, :].argmax(axis=-1)
  class_list += [species_list[y_class]]

submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

submission.to_csv('submission.csv', index=False)

print('Submission file generated. All done.')